### Evidencia de aprendizaje 1. Análisis y herramientas de extracción de datos

#### Ossman Mejía
#### Olimpo Macea
#### Grupo: PREICA2402B020101
#### Andrés Felipe Palacio Angel
####  Programación para Análisis de Datos
#### Noviembre 2024

#### Introducción

El scraping es una útil herramienta para extracción de datos específicos de forma automática. esto con el objetivo de monitorear alguna variable en específico, para recolectar datos sobre el mercado o competidores o para tener de manera más simple una lista de productos que de otra llevaría más tiempo poder extraer o analizar.

Con tal propósito en mente se realiza la extracción de datos sobre tarjetas gráficas para pc de una de las plataformas de compra en línea más grade del continente como lo es mercado libre esto con el objetivo de analizar referencias y precios que hay en esta plataforma y tener una lista de referencia para compras futuras.

La metodología empleada es Beautifulsoup por su facilidad de uso teniendo en cuenta los requerimientos de la actividad y la consulta a realizar.

### Descrpición General

##### página y artículo

La página para el desarrollo de la actividad será una de las sugeridas en la actividad en este caso MercadoLibre, ya proporciona datos de precios locales del articulo a consultar sin tener que realizar un análisis adicional por tasas de cambio, lo cual nos pareció más práctico para realizar la extracción de los datos.

##### ¿por qué deseas analizar este artículo y la empresa de comercio?

Las tarjetas gráficas son hoy día una parte fundamental para cualquier equipo de cómputo done se realicen actividades de diseño, minería de datos y gamming por lo que es un componente clave para cualquier persona que se desempeñe en estos campos. Tomando esto como base se desea tener una lista de las diferentes tarjetas gráficas que se encuentran en el mercado publicadas en mercado libre ya que es una excelente fuente de referencia a tener en cuenta con otros proveedores de estos mismos componentes. 

Realizar una extracción de estos datos y poder disponer de ellos de forma ágil y precisa facilita mucho la buena toma de decisiones con respecto a cuál sería la mejor opción costo beneficio del mercado tomando como base los resultados obtenidos de una de las principales páginas de compras online.


##### 

#### Metodología empleada de scraping.

Este código realiza una extracción de información de productos en Mercado Libre, específicamente busca el título y el precio de los productos listados en una página de búsqueda. Primero, envía una solicitud a la URL especificada y obtiene el HTML de la página. Luego, analiza el contenido HTML utilizando BeautifulSoup para localizar los elementos que contienen la información de los productos. Dentro de cada elemento de producto, intenta identificar el título y el precio utilizando clases específicas (CSS selectors).

Para cada producto encontrado, guarda el título y el precio en una lista de diccionarios, donde cada diccionario representa un producto con sus datos. Además, guarda una copia completa del HTML de la página en una ubicación especificada en el sistema de archivos, creando el directorio si no existe. Si hay errores en el proceso, maneja los casos de forma apropiada, devolviendo mensajes de error si no se puede acceder a la URL o si el HTML no se puede analizar correctamente.




In [8]:
import requests
from bs4 import BeautifulSoup
import os
import csv


In [11]:
def guardar_en_csv(resultados):
    """
    Guarda la lista de resultados en un archivo CSV.
    
    Args:
        resultados: La lista de diccionarios con la información de los productos.
    """
    if not resultados:  # Si no hay datos para guardar
        print("No hay resultados para guardar.")
        return

    relative_path = os.path.join(os.getcwd(), 'actividad1')
    if not os.path.exists(relative_path):
        os.makedirs(relative_path)  # Crear el directorio si no existe

    # Ruta completa donde se guardará el archivo CSV
    archivo_csv = os.path.join(relative_path, 'productos.csv')
    print(f"Guardando el archivo CSV en: {archivo_csv}")  # Muestra la ruta

    with open(archivo_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Título", "Precio"])
        writer.writeheader()  # Escribe la cabecera del archivo CSV
        writer.writerows(resultados)  # Escribe los resultados

# Función para raspar Mercado Libre


def scrape_mercadolibre(url):
    """
    Extrae información de los productos más vendidos en Mercado Libre.

    Args:
        url: La URL de la página de Mercado Libre a raspar.

    Returns:
        Una lista de diccionarios con la información extraída o un mensaje de error.
    """
    try:
        # 1. Obtener el contenido HTML de la página
        response = requests.get(url)
        response.raise_for_status()

        # 2. Analizar el HTML con BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # 3. Encontrar los elementos que contienen la información de los productos.
        productos = soup.find_all("li", class_="ui-search-layout__item")

        resultados = []
        for producto in productos:
            # Encuentra el título del producto
            titulo_elemento = producto.find(
                "h2", class_="poly-box poly-component__title")
            titulo = titulo_elemento.text.strip() if titulo_elemento else "Título no encontrado"

            # Encuentra el precio del producto
            precio_elemento = producto.find(
                "div", class_="poly-price__current")
            precio = precio_elemento.text.strip() if precio_elemento else "Precio no encontrado"

            # Imprimir para verificar que los productos se están extrayendo correctamente
            print(f"Título: {titulo}, Precio: {precio}")

            resultados.append({"Título": titulo, "Precio": precio})

        # Si hay resultados, se guardan en CSV
        if resultados:
            guardar_en_csv(resultados)
        else:
            print("No se encontraron productos.")

        # 4. Guardar el HTML en el directorio especificado
        relative_path = os.path.join(os.getcwd(), 'actividad1')
        if not os.path.exists(relative_path):
            os.makedirs(relative_path)
        with open(os.path.join(relative_path, 'pagina.html'), "w", encoding="utf-8") as f:
            f.write(str(soup))  # Guarda el HTML completo

        return resultados

    except requests.exceptions.RequestException as e:
        return f"Error al obtener la URL: {e}"
    except AttributeError as e:
        return f"Error al analizar el HTML: {e}. Verifica el selector CSS."
    except Exception as e:
        return f"Error inesperado: {e}"


# URL de búsqueda de tarjetas gráficas en Mercado Libre Colombia
url_mercadolibre = "https://listado.mercadolibre.com.co/tarjetas-graficas"

resultados = scrape_mercadolibre(url_mercadolibre)

Título: Tarjeta Gráfica De Escritorio Rx580 De 8 Gb, Ddr5, 256 Bits,, Precio: $980.448
Título: Tarjeta Gráfica Gtx550ti 6gb Gpu 192 Bits, Compatible Con Vi, Precio: $534.241
Título: Tarjeta de video Nvidia Gigabyte  Aero GeForce RTX 40 Series RTX 4060 8 GB GV-N4060AERO OC-8GD OC Edition 8GB, Precio: $1.697.90021% OFF
Título: Tarjeta de video Nvidia Gigabyte  GeForce 700 Series GT 710 GV-N710D3-2GL (REV 2.0) 2GB, Precio: $255.5505% OFF
Título: Tarjeta De Video Gigabyte GT 1030 Low Profile D4 2G, Precio: $348.158
Título: Placa De Video Gigabyte Rtx 3050 Windforce Oc 6gb Nvidia 3c, Precio: $949.90038% OFF
Título: Tarjeta Gráfica Para Computadora Gddr5 Rx580 Game 8 Gb 256 B, Precio: $477.711
Título: Tarjeta De Video Gráfica Nvidia Quadro K1200 4gb, Precio: $720.000
Título: Tarjeta Grafica Gigabyte Nvidia Geforce Gt 710 2 Gb, Precio: $295.000
Título: Tarjeta de video AMD ASRock  Phantom Gaming Radeon RX 500 Series RX 550 PHANTOM G R RX550 2G 2GB, Precio: $413.000
Título: Tarjetas Gráficas R

#### Resultados y conclusiones

Luego de obtener las listas de los productos y sus precios podemos sacar varias conclusiones de los resultados obtenidos:

Variedad de Productos: La lista de productos obtenidos muestra una variedad de tarjetas gráficas de diferentes marcas y modelos (NVIDIA y AMD), lo que indica que hay una oferta amplia que puede satisfacer tanto a usuarios que buscan tarjetas de gama media como a aquellos interesados en productos de gama alta.

Rango de Precios: Los precios de las tarjetas gráficas oscilan desde aproximadamente $800,000 hasta $2,200,000. Esto sugiere que hay opciones para diferentes presupuestos y necesidades de rendimiento, aunque las tarjetas de gama alta como las de la serie RTX de NVIDIA tienden a ser más costosas.

Tendencia del Mercado: Observando los productos extraídos, es evidente que los modelos de tarjetas gráficas más recientes, como la serie RTX de NVIDIA, son significativamente más caros que los modelos anteriores, como la GTX 1050 Ti. 


#### Bibliografia

Galperin, M. (s.f.). Mercado libre. Obtenido de Galperin, M. (s.f.). Mercado libre. Obtenido de https://www.mercadolibre.com.co/


Andrade, F. (s.f.). youtube. Obtenido de https://www.youtube.com/watch?v=kPNHKrOqedI